## Analysis of the Aug 2023 primary results for the Bellingham mayoral race in Whatcom County

# Background Info

Good source for graph inspiration
https://medium.com/analytics-vidhya/plotly-for-geomaps-bb75d1de189f

Used https://plotly.com/python/choropleth-maps "Choropleth map using GeoJSON" for source of how-to

1. Get shape files of Precincts from Whatcom County https://documents.co.whatcom.wa.us/WebLink/Browse.aspx?id=2951672&dbid=100&repo=WC

2. Convert shape zip file to geojson using mapshaper.org

*   Drag and drop shape zip onto collab via Files tab
*   Open Console and type mapshaper -proj wg84 to convert coordinates to lat/long
*   Export to geojson
*   Note this json file does not have id tag which is needed to link Precinct to external csv file.  Workaroun is to use parameter 'featureidkey' to explicitly specify id

3. Import csv and json data to collab or link via drive


# Data Prep Instructions
1. Get shape files of Precincts from Whatcom County https://documents.co.whatcom.wa.us/WebLink/Browse.aspx?id=2951672&dbid=100&repo=WC

2. Convert shape zip file to geojson using mapshaper.org

* Drag and drop shape zip onto mapshaper site
* Open Console and type mapshaper -proj wg84 to convert coordinates to lat/long
* Export to geojson

  Note this json file does not have id tag which is needed to link Precinct to external csv file. Workaroun is to use parameter 'featureidkey' to explicitly specify id

3. Import csv and json data to collab or link via drive

## Import the raw data into dataframes

In [ ]:
import pandas as pd

#df_votes_by_precinct = pd.read_csv('/content/20230801_whatcom_123167-precincts.csv')
df_votes_by_precinct = pd.read_csv('/content/20230801_whatcom_123167-precincts.csv', skiprows=1)

#Remove the totals in row 1
df_votes_by_precinct = df_votes_by_precinct.drop(0)
#Add precinct name to first column
df_votes_by_precinct.rename(columns={'Unnamed: 0': 'Precinct'}, inplace=True)

#Add precinct totals column
df_votes_by_precinct['Totals By Precinct'] = df_votes_by_precinct.iloc[:, 1:].sum(axis=1)

df_votes_by_precinct

## Add calculated columns

In [ ]:
df_votes_by_precinct['Percent Seth Votes'] = (df_votes_by_precinct['Seth Fleetwood'] / df_votes_by_precinct['Totals By Precinct']) * 100
df_votes_by_precinct['Percent Kim Votes'] = (df_votes_by_precinct['Kim Lund'] / df_votes_by_precinct['Totals By Precinct']) * 100


df_votes_by_precinct['Percent SF-KL'] = ((df_votes_by_precinct['Seth Fleetwood'] - df_votes_by_precinct['Kim Lund'])/ df_votes_by_precinct['Totals By Precinct']) * 100

df_votes_by_precinct['Color Group'] = pd.cut(df_votes_by_precinct['Percent SF-KL'], bins=[-100, -0.5, 15, 100], labels=['Kim', 'Neutral', 'Seth'])

#df_votes_by_precinct['Kim Lund Best'] = (df_votes_by_precinct['Kim Lund'] + df_votes_by_precinct['Kristina Michele Martens'] + df_votes_by_precinct['Christopher J. McCoy']/2 + df_votes_by_precinct['Mike McAuley']/2 + df_votes_by_precinct['WRITE-IN'])
df_votes_by_precinct['Kim Lund Best'] = (df_votes_by_precinct['Kim Lund'] + df_votes_by_precinct['Kristina Michele Martens'])
df_votes_by_precinct['Percent Kim Best Votes'] = (df_votes_by_precinct['Kim Lund Best'] / df_votes_by_precinct['Totals By Precinct']) * 100
df_votes_by_precinct['Percent SF-KL Best'] = ((df_votes_by_precinct['Seth Fleetwood'] - df_votes_by_precinct['Kim Lund Best'])/ df_votes_by_precinct['Totals By Precinct']) * 100

df_votes_by_precinct.to_csv('/content/20230801_whatcom_123167-precincts-processed.csv')
df_votes_by_precinct
#df_votes_by_precinct.dtypes

In [ ]:
import json

# File path to the GeoJSON file
# First upload json file to colab storage
precinct_file = 'April2023_Precinct_Splits_wgs84.json'

# Reading the JSON file
with open(precinct_file, 'r') as file:
    precinct_data = json.load(file)

# Displaying the JSON data features dictionary for each precinct
print(precinct_data)
precinct_data["features"][0]

# Within features is a properties dictionary which contains the precinct id
print(precinct_data["features"][0]["properties"])

## Setup Graph with integrated precinct data

In [ ]:
import plotly.express as px

# Create the choropleth map
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color=None,
                           locations="Precinct", featureidkey="properties.Precinct",
                           labels="Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10, opacity=0.1)

# Adjust the marker properties for boundaries
for trace in fig.data:
    trace.marker.line.width = 6
    trace.marker.line.color = 'red'

# Update layout
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Plot Kim Primary Raw Vote Results

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Kim Lund",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
### Plot Seth Primary Raw Vote Results

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Seth Fleetwood",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Plot Seth Percent of Total

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Percent Seth Votes",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Plot Kim Percent of Total

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Percent Kim Votes",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#0-49.9 = Red
#50-100 = Green
df_votes_by_precinct['Color Group Best'] = pd.cut(df_votes_by_precinct['Percent Kim Best Votes'], bins=[0, 49.9, 100], labels=['Lost', 'Won'])

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Color Group Best",
                           locations="Precinct", featureidkey="properties.Precinct",
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Color Group",
                           color_discrete_map={'Kim':'blue', 'Seth':'orange', 'Neutral': 'green'},
                           locations="Precinct", featureidkey="properties.Precinct",
                           opacity=0.6,
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#0-49.9 = Red
#50-100 = Green
df_votes_by_precinct['Color Group Best Diff'] = pd.cut(df_votes_by_precinct['Percent SF-KL Best'], bins=[-100, 0, 100], labels=['Won', 'Lost'])

In [ ]:
fig = px.choropleth_mapbox(df_votes_by_precinct, geojson=precinct_data, color="Color Group Best Diff",
                           color_discrete_map={'Kim':'blue', 'Seth':'orange', 'Neutral': 'green'},
                           locations="Precinct", featureidkey="properties.Precinct",
                           opacity=0.6,
                           center={"lat": 48.7519, "lon": -122.4787},
                           mapbox_style="carto-positron", zoom=10)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()